<a href="https://colab.research.google.com/github/antwoor/ds_lessons/blob/master/cats_and_dogs_vgg16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Распознавание собак и кошек на изображениях с помощью предварительно обученной нейронной сети VGG16

**Источник данных** - соревнования Kaggle [Dogs vs. Cats](https://www.kaggle.com/c/dogs-vs-cats/data).

Для распознавания используется предварительно обученная сверточная нейронная сеть VGG16.

Перед использованием необходимо скачать и подготовить данные для обучения, проверки и тестирования.

# Подготовка данных для распознавания котов и собак на изображениях

**Источник данных** - соревнования Kaggle [Dogs vs. Cats](https://www.kaggle.com/c/dogs-vs-cats/data).

Пример подготовки данных для обучения нейронной сети на Keras. Данные разбиваются на три каталога:
- train (данные для обучения)
- val (данные для проверки)
- test (данные для тестирования)

В каждом каталоге создаются по два подкаталога, в соответсвии с названиями классов: cats и dogs.

Изображения переписваются из исходного каталога в новую структуру. По-умолчанию для обучения используется 70% изображений, для проверки - 15%, для тестрования также 15%.

Перед запуском необходимо скачать файл train.zip с набором изображений кошек и собак с сайта соревнования Kaggle [Dogs vs. Cats](https://www.kaggle.com/c/dogs-vs-cats/data) и распаковать его.

In [ ]:
!pip install patool
import patoolib
patoolib.extract_archive("train.rar", outdir="/content")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.5/77.5 kB 3.0 MB/s eta 0:00:00
patool: Extracting train.rar ...
patool: running /usr/bin/unrar x -- /content/train.rar
patool:     with cwd='/content'
patool: ... train.rar extracted to `/content'.


'/content'

In [ ]:
import shutil
import os

In [ ]:
# Каталог с набором данных
data_dir = '/content/train'
# Каталог с данными для обучения
train_dir = 'train'
# Каталог с данными для проверки
val_dir = 'val'
# Каталог с данными для тестирования
test_dir = 'test'
# Часть набора данных для тестирования
test_data_portion = 0.15
# Часть набора данных для проверки
val_data_portion = 0.15
# Количество элементов данных в одном классе
nb_images = 313

Функция создания каталога с двумя подкаталогами по названию классов: cats и dogs

In [ ]:
def create_directory(dir_name):
    if os.path.exists(dir_name):
        shutil.rmtree(dir_name)
    os.makedirs(dir_name)
    os.makedirs(os.path.join(dir_name, "cats"))
    os.makedirs(os.path.join(dir_name, "dogs"))

Создание структуры каталогов для обучающего, проверочного и тестового набора данных

In [ ]:
create_directory(train_dir)
create_directory(val_dir)
create_directory(test_dir)

Функция копирования изображений в заданный каталог. Изображения котов и собак копируются в отдельные подкаталоги

In [ ]:
os.rename("text.txt", "renamed-text.txt")

In [ ]:
def copy_images(start_index, end_index, source_dir, dest_dir):
    for i in range(start_index, end_index):
        shutil.copy2(os.path.join(source_dir, "cat." + str(i) + ".jpg"),
                    os.path.join(dest_dir, "cats"))
        shutil.copy2(os.path.join(source_dir, "dog." + str(i) + ".jpg"),
                   os.path.join(dest_dir, "dogs"))

Расчет индексов наборов данных для обучения, приверки и тестирования

In [ ]:
start_val_data_idx = int(nb_images * (1 - val_data_portion - test_data_portion))
start_test_data_idx = int(nb_images * (1 - test_data_portion))
print(start_val_data_idx)
print(start_test_data_idx)


219
266


Копирование изображений

In [ ]:
copy_images(0, start_val_data_idx, data_dir, train_dir)
copy_images(start_val_data_idx, start_test_data_idx, data_dir, val_dir)
copy_images(start_test_data_idx, nb_images, data_dir, test_dir)

FileNotFoundError: ignored

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.applications import VGG16
from keras.optimizers import Adam

In [ ]:
# Каталог с данными для обучения
train_dir = 'train'
# Каталог с данными для проверки
val_dir = 'val'
# Каталог с данными для тестирования
test_dir = 'test'
# Размеры изображения
img_width, img_height = 150, 150
# Размерность тензора на основе изображения для входных данных в нейронную сеть
# backend Tensorflow, channels_last
input_shape = (img_width, img_height, 3)
# Размер мини-выборки
batch_size = 64
# Количество изображений для обучения
nb_train_samples = 17500
# Количество изображений для проверки
nb_validation_samples = 3750
# Количество изображений для тестирования
nb_test_samples = 3750

## Загружаем предварительно обученную нейронную сеть

In [ ]:
vgg16_net = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

58889256/58889256 [==============================] - 3s 0us/step


"Замораживаем" веса предварительно обученной нейронной сети VGG16

In [ ]:
vgg16_net.trainable = False

In [ ]:
vgg16_net.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 37, 37, 128)       0     

## Создаем составную нейронную сеть на основе VGG16

In [ ]:
model = Sequential()
# Добавляем в модель сеть VGG16 вместо слоя
model.add(vgg16_net)
model.add(Flatten())
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 8192)              0         
                                                                 
 dense (Dense)               (None, 256)               2097408   
                                                                 
 activation (Activation)     (None, 256)               0         
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 257       
                                                                 
 activation_1 (Activation)   (None, 1)                 0

Компилируем составную нейронную сеть

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer=Adam(lr=1e-5),
              metrics=['accuracy'])

/usr/local/lib/python3.9/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


## Создаем генератор изображений

Генератор изображений создается на основе класса ImageDataGenerator. Генератор делит значения всех пикселов изображения на 255.

In [ ]:
datagen = ImageDataGenerator(rescale=1. / 255)


Генератор данных для обучения на основе изображений из каталога

In [ ]:
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

FileNotFoundError: ignored

Генератор данных для проверки на основе изображений из каталога

In [ ]:
val_generator = datagen.flow_from_directory(
    val_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')


Found 3750 images belonging to 2 classes.


Генератор данных для тестирования на основе изображений из каталога

In [ ]:
test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 3750 images belonging to 2 classes.


## Обучаем модель с использованием генераторов

train_generator - генератор данных для обучения

validation_data - генератор данных для проверки

In [ ]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=10,
    validation_data=val_generator,
    validation_steps=nb_validation_samples // batch_size)

Epoch 1/10
273/273 [==============================] - 462s - loss: 0.2524 - acc: 0.8857 - val_loss: 0.1329 - val_acc: 0.9472
Epoch 2/10
273/273 [==============================] - 458s - loss: 0.0975 - acc: 0.9630 - val_loss: 0.1237 - val_acc: 0.9512
Epoch 3/10
273/273 [==============================] - 454s - loss: 0.0606 - acc: 0.9762 - val_loss: 0.0804 - val_acc: 0.9710
Epoch 4/10
273/273 [==============================] - 454s - loss: 0.0446 - acc: 0.9840 - val_loss: 0.0791 - val_acc: 0.9696
Epoch 5/10
273/273 [==============================] - 454s - loss: 0.0254 - acc: 0.9915 - val_loss: 0.0976 - val_acc: 0.9699
Epoch 6/10
273/273 [==============================] - 453s - loss: 0.0169 - acc: 0.9942 - val_loss: 0.0883 - val_acc: 0.9693
Epoch 7/10
273/273 [==============================] - 451s - loss: 0.0125 - acc: 0.9960 - val_loss: 0.0702 - val_acc: 0.9740
Epoch 8/10
273/273 [==============================] - 452s - loss: 0.0146 - acc: 0.9951 - val_loss: 0.1078 - val_acc: 0.9704


## Оцениваем качество работы сети с помощью генератора

In [ ]:
scores = model.evaluate_generator(test_generator, nb_test_samples // batch_size)

In [ ]:
print("Аккуратность на тестовых данных: %.2f%%" % (scores[1]*100))

Аккуратность на тестовых данных: 97.31%


# Тонкая настройка сети (fine tuning)

"Размораживаем" последний сверточный блок сети VGG16

In [ ]:
vgg16_net.trainable = True
trainable = False
for layer in vgg16_net.layers:
    if layer.name == 'block5_conv1':
        trainable = True
    layer.trainable = trainable

In [ ]:
# Проверяем количество обучаемых параметров
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               2097408   
_________________________________________________________________
activation_1 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
_________________________________________________________________
activation_2 (Activation)    (None, 1)                 0         
Total para

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer=Adam(lr=1e-5),
              metrics=['accuracy'])

In [ ]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=2,
    validation_data=val_generator,
    validation_steps=nb_validation_samples // batch_size)

Epoch 1/2
273/273 [==============================] - 184s - loss: 3.6300e-04 - acc: 0.9999 - val_loss: 0.1374 - val_acc: 0.9734
Epoch 2/2
273/273 [==============================] - 184s - loss: 4.0974e-04 - acc: 0.9999 - val_loss: 0.1476 - val_acc: 0.9759


In [ ]:
scores = model.evaluate_generator(test_generator, nb_test_samples // batch_size)
print("Точность на тестовых данных: %.2f%%" % (scores[1]*100))

Аккуратность на тестовых данных: 97.15%
